In [19]:
import pandas as pd
import numpy as np

In [20]:
pivot_data4 = pd.read_csv('/Users/emilkoch/Desktop/2Tango/Data Files/Dataset_2_glasses/pivot_data4.csv')
activity_type_full = pd.read_csv('/Users/emilkoch/Desktop/2Tango/Data Files/Dataset_2_glasses/report_full_activity_type.csv')
smartphone = pd.read_csv('/Users/emilkoch/Desktop/2Tango/Data Files/Dataset_2_glasses/pivot_table_s_split.csv')

/var/folders/4r/5qs7726s3_dc54vtgh1sc2740000gn/T/ipykernel_85916/1207623996.py:3: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  smartphone = pd.read_csv('/Users/emilkoch/Desktop/2Tango/Data Files/Dataset_2_glasses/pivot_table_s_split.csv')


In [22]:
import pandas as pd
import numpy as np

def classify_motion_row(row):
    # Extract relevant data from the row
    accel_x = row['accel_x']
    accel_y = row['accel_y']
    accel_z = row['accel_z']
    gyro_x = row['gyro_x']
    gyro_y = row['gyro_y']
    gyro_z = row['gyro_z']
    orientation_x = row['orientation_x']
    orientation_y = row['orientation_y']
    orientation_z = row['orientation_z']
    
    # Classify motion based on accelerometer and gyroscope readings
    if np.any((accel_x != 0) | (accel_y != 0) | (accel_z != 0)):
        # Check if all gyroscope readings are zero
        if not np.any((gyro_x != 0) | (gyro_y != 0) | (gyro_z != 0)):
            # Check orientation for refining moving motion
            if orientation_z > 90 or orientation_z < -90:
                motion_type = "Vertical Movement"
            elif orientation_x > 90 or orientation_x < -90:
                motion_type = "Horizontal Movement"
            else:
                motion_type = "Moving"
        else:
            # If gyroscope readings are not all zero, classify as "Turning/Tilting"
            motion_type = "Turning/Tilting"
    elif np.any((gyro_x != 0) | (gyro_y != 0) | (gyro_z != 0)):
        # If accelerometer readings are all zero, check gyroscope readings
        motion_type = "Turning/Tilting"
    else:
        # If both accelerometer and gyroscope readings are zero, classify as "Stationary"
        motion_type = "Stationary"

    return pd.Series([motion_type])

# Apply the function row-wise to the dataframe
pivot_data4['motion_type'] = pivot_data4.apply(classify_motion_row, axis=1)


In [23]:
from sklearn.preprocessing import LabelEncoder

# Create and fit the LabelEncoder
label_encoder_motion = LabelEncoder()
pivot_data4['motion_encoded'] = label_encoder_motion.fit_transform(pivot_data4['motion_type'])

In [19]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential, save_model
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import classification_report
from sklearn.utils.class_weight import compute_class_weight
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras import regularizers
import numpy as np

# Split the data into features (X) and target variable (y)
X = pivot_data4[['accel_x', 'accel_y', 'accel_z', 'gyro_x', 'gyro_y', 'gyro_z']]
y = pivot_data4['motion_encoded']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Further split the training data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

# Standardize the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

# Define the model with L1 regularization
model = Sequential([
    Dense(64, activation='relu', kernel_regularizer=regularizers.l1(0.001), input_shape=(X_train_scaled.shape[1],)),
    Dropout(0.5),
    BatchNormalization(),
    Dense(64, activation='relu', kernel_regularizer=regularizers.l1(0.001)),
    Dropout(0.5),
    BatchNormalization(),
    Dense(len(np.unique(pivot_data4['motion_encoded'])), activation='softmax')
])

# Compile the model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Encode class labels
le = LabelEncoder()
y_train_encoded = le.fit_transform(y_train)

# Compute class weights using the encoded labels
class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(y_train_encoded), y=y_train_encoded)

# Create a class weight dictionary
class_weight_dict = dict(zip(le.transform(le.classes_), class_weights))

# Define early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=5)

# Train the model with class weights and early stopping
history = model.fit(X_train_scaled, y_train_encoded, epochs=20, validation_data=(X_val_scaled, y_val), class_weight=class_weight_dict, callbacks=[early_stopping])

# Save the model
file_path = "/Users/emilkoch/Desktop/2Tango/messenger/research/Model_Save/motion.keras"
save_model(model, file_path)

# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(X_test_scaled, y_test)
print("Test Accuracy:", test_accuracy)

# Make predictions
y_pred_prob = model.predict(X_test_scaled)
y_pred = np.argmax(y_pred_prob, axis=1)

# Calculate precision, recall, and F1 score
report = classification_report(y_test, y_pred)
print(report)

Epoch 1/20
3474/3474 [==============================] - 15s 4ms/step - loss: 1.8236 - accuracy: 0.6960 - val_loss: 0.6908 - val_accuracy: 0.9406
Epoch 2/20
3474/3474 [==============================] - 14s 4ms/step - loss: 1.2233 - accuracy: 0.8680 - val_loss: 0.6342 - val_accuracy: 0.9385
Epoch 3/20
3474/3474 [==============================] - 14s 4ms/step - loss: 1.0585 - accuracy: 0.8914 - val_loss: 0.5671 - val_accuracy: 0.9377
Epoch 4/20
3474/3474 [==============================] - 13s 4ms/step - loss: 0.9513 - accuracy: 0.9069 - val_loss: 0.5453 - val_accuracy: 0.9326
Epoch 5/20
3474/3474 [==============================] - 13s 4ms/step - loss: 0.8861 - accuracy: 0.9167 - val_loss: 0.5040 - val_accuracy: 0.9396
Epoch 6/20
3474/3474 [==============================] - 13s 4ms/step - loss: 0.8799 - accuracy: 0.9237 - val_loss: 0.4559 - val_accuracy: 0.9417
Epoch 7/20
3474/3474 [==============================] - 12s 4ms/step - loss: 0.8554 - accuracy: 0.9293 - val_loss: 0.5596 - val_ac

In [11]:
def classify_activity(row):
    motion_type = row['motion_type']
    if motion_type == "Vertical Movement":
        predicted_activity = ["Running", "Walking", "Walking & party", "Football"]
    elif motion_type == "Horizontal Movement":
        predicted_activity = ["In vehicle", "Train", "In Bus", "On Bus stop"]
    elif motion_type == "Moving":
        predicted_activity = ["Cooking", "Meeting", "Movie", "Shopping & wearing"]
    elif motion_type == "Turning/Tilting":
        predicted_activity = ["Phone was out of the pocket (forgot)", "Took off glasses"]
    elif motion_type == "Stationary":
        predicted_activity = ["At home", "Pause", "Shop", "Work", "Eating", "Sleeping", "Watching TV", "Eating", "Picnic", "Video Games"]
    return predicted_activity

# Assuming 'pivot_data4' is your DataFrame
pivot_data4['predicted_activity'] = pivot_data4.apply(classify_activity, axis=1)

In [24]:
pivot_data4

,timestamp,step_counter,step_detector,accel_x,accel_y,accel_z,gyro_x,gyro_y,gyro_z,linear_accel_x,...,orientation_x,orientation_y,orientation_z,rotation_scalar,rotation_i,rotation_j,rotation_k,rotation_angle,motion_type,motion_encoded
0,2017-06-29 07:45:56.262,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Stationary,2
1,2017-06-29 07:46:01.263,0.0,0.0,-0.344086,0.683304,9.967361,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Moving,1
2,2017-06-29 07:46:01.324,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Stationary,2
3,2017-06-29 07:46:01.329,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-0.018069,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Stationary,2
4,2017-06-29 07:46:01.402,0.0,0.0,0.000000,0.000000,0.000000,0.004166,-0.013275,0.006165,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Turning/Tilting,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
173657,2017-07-13 20:21:35.009,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Stationary,2
173658,2017-07-13 20:21:35.020,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Stationary,2
173659,2017-07-13 20:21:35.060,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Stationary,2
173660,2017-07-13 20:21:35.124,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Stationary,2


In [7]:
activity_types = activity_type_full['activity_type'].unique()

In [8]:
activity_types

array(['Video games', 'In computer', 'At home', 'Eat', 'Sleep',
       'Watching TV', 'In vehicle', 'Picnic ', 'Walk', 'Running',
       'Phone was out of the pocket (forgot)', 'Train', 'Cooking',
       'In bus', 'Meeting', 'Pause', 'Movie', 'Football', 'Shop',
       'Shopping& wearing', 'Walking&party', 'On bus stop',
       'Took off glasses', 'Work'], dtype=object)

In [25]:
pivot_data4_pathway = '/Users/emilkoch/Desktop/2Tango/Data Files/Dataset_2_glasses/pivot_data4.csv'

pivot_data4.to_csv(pivot_data4_pathway, index=False)

In [5]:
pivot_data4['motion_type'].value_counts()


motion_type
Stationary             156088
Turning/Tilting          8986
Moving                   7795
Vertical Movement         502
Horizontal Movement       291
Name: count, dtype: int64

This diversity of motion_types shows that the function is working. 

In [26]:
# Assuming 'pivot_data4' is your DataFrame containing the timestamp column
pivot_data4['timestamp'] = pd.to_datetime(pivot_data4['timestamp'])

# Extract hour and minute from the timestamp and convert to minutes
pivot_data4['timestamp'] = pivot_data4['timestamp'].dt.hour * 60 + pivot_data4['timestamp'].dt.minute

In [27]:
pivot_data4

,timestamp,step_counter,step_detector,accel_x,accel_y,accel_z,gyro_x,gyro_y,gyro_z,linear_accel_x,...,orientation_x,orientation_y,orientation_z,rotation_scalar,rotation_i,rotation_j,rotation_k,rotation_angle,motion_type,motion_encoded
0,465,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Stationary,2
1,466,0.0,0.0,-0.344086,0.683304,9.967361,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Moving,1
2,466,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Stationary,2
3,466,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-0.018069,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Stationary,2
4,466,0.0,0.0,0.000000,0.000000,0.000000,0.004166,-0.013275,0.006165,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Turning/Tilting,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
173657,1221,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Stationary,2
173658,1221,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Stationary,2
173659,1221,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Stationary,2
173660,1221,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Stationary,2


The logic I have now is that smartphone and smartwatch etc. indeed do not have been used simultaneously. Regardless, we may get some insights regarding the timing of activities and filter accordingly when excluding day and year. 

In [28]:
smartphone = smartphone.drop(columns = ['activity_type'])

In [29]:
# Convert timestamp columns to datetime objects
activity_type_full['timestamp'] = pd.to_datetime(activity_type_full['timestamp'])
smartphone['timestamp'] = pd.to_datetime(smartphone['timestamp'])

In [30]:
# Convert timestamp columns to datetime objects
activity_type_full['timestamp'] = pd.to_datetime(activity_type_full['timestamp'])
smartphone['timestamp'] = pd.to_datetime(smartphone['timestamp'])

# Group activity_type_full by timestamp and transform it to align with smartphone
activity_type_grouped = activity_type_full.groupby(smartphone['timestamp'])['activity_type'].apply(lambda x: ','.join(x)).reset_index()

# Merge activity_type_grouped with smartphone based on timestamp
smartphone = pd.merge(smartphone, activity_type_grouped, on='timestamp', how='left')

# Print or further process the updated DataFrame
print(smartphone)


                     timestamp  step_detector_values  audio_value_1  \
0      2017-06-29 07:45:16.506                   1.0            0.0   
1      2017-06-29 07:45:17.407                   0.0            0.0   
2      2017-06-29 07:45:18.407                   0.0          227.0   
3      2017-06-29 07:45:19.407                   0.0          590.0   
4      2017-06-29 07:45:20.446                   0.0         1724.0   
...                        ...                   ...            ...   
617948 2017-07-13 20:13:33.671                   0.0         1956.0   
617949 2017-07-13 20:13:34.663                   0.0         2673.0   
617950 2017-07-13 20:13:35.663                   0.0         2340.0   
617951 2017-07-13 20:13:36.668                   0.0         2046.0   
617952 2017-07-13 20:13:37.668                   0.0         2441.0   

        audio_value_2  audio_value_3  audio_value_4 activity_type  
0             0.00000            0.0            0.0   Video games  
1          

In [31]:
smartphone.fillna(0, inplace=True)

In [32]:
# Filter the DataFrame to keep rows where "activity_type" is not equal to 0
merged_data_s_f = smartphone[smartphone['activity_type'] != 0]

In [33]:
merged_data_s_f

,timestamp,step_detector_values,audio_value_1,audio_value_2,audio_value_3,audio_value_4,activity_type
0,2017-06-29 07:45:16.506,1.0,0.0,0.00000,0.0,0.0,Video games
1,2017-06-29 07:45:17.407,0.0,0.0,0.00000,0.0,32767.0,In computer
2,2017-06-29 07:45:18.407,0.0,227.0,113.50000,227.0,227.0,At home
3,2017-06-29 07:45:19.407,0.0,590.0,272.33334,590.0,227.0,In computer
4,2017-06-29 07:45:20.446,0.0,1724.0,635.25000,1724.0,227.0,Eat
...,...,...,...,...,...,...,...
146,2017-06-29 07:47:36.448,0.0,23886.0,4708.92140,32767.0,45.0,Work
147,2017-06-29 07:47:37.448,0.0,7923.0,4731.71630,32767.0,45.0,Walk
148,2017-06-29 07:47:38.448,0.0,7005.0,4747.72560,32767.0,45.0,Train
149,2017-06-29 07:47:39.449,0.0,1085.0,4722.11200,32767.0,45.0,Walk


In [34]:
# Assuming 'pivot_data4' is your DataFrame containing the timestamp column
merged_data_s_f['timestamp'] = pd.to_datetime(merged_data_s_f['timestamp'])

# Extract hour and minute from the timestamp and convert to minutes
merged_data_s_f['timestamp'] = merged_data_s_f['timestamp'].dt.hour * 60 + merged_data_s_f['timestamp'].dt.minute


/var/folders/4r/5qs7726s3_dc54vtgh1sc2740000gn/T/ipykernel_85916/3425056289.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_data_s_f['timestamp'] = pd.to_datetime(merged_data_s_f['timestamp'])
/var/folders/4r/5qs7726s3_dc54vtgh1sc2740000gn/T/ipykernel_85916/3425056289.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_data_s_f['timestamp'] = merged_data_s_f['timestamp'].dt.hour * 60 + merged_data_s_f['timestamp'].dt.minute


In [35]:
merged_data_s_f

,timestamp,step_detector_values,audio_value_1,audio_value_2,audio_value_3,audio_value_4,activity_type
0,465,1.0,0.0,0.00000,0.0,0.0,Video games
1,465,0.0,0.0,0.00000,0.0,32767.0,In computer
2,465,0.0,227.0,113.50000,227.0,227.0,At home
3,465,0.0,590.0,272.33334,590.0,227.0,In computer
4,465,0.0,1724.0,635.25000,1724.0,227.0,Eat
...,...,...,...,...,...,...,...
146,467,0.0,23886.0,4708.92140,32767.0,45.0,Work
147,467,0.0,7923.0,4731.71630,32767.0,45.0,Walk
148,467,0.0,7005.0,4747.72560,32767.0,45.0,Train
149,467,0.0,1085.0,4722.11200,32767.0,45.0,Walk


In [36]:
merged_data_s_f_filtered = merged_data_s_f[['timestamp', 'activity_type']]

In [37]:
merged_data_s_f_filtered 

,timestamp,activity_type
0,465,Video games
1,465,In computer
2,465,At home
3,465,In computer
4,465,Eat
...,...,...
146,467,Work
147,467,Walk
148,467,Train
149,467,Walk


In [38]:
# Merge the dataframes on the 'timestamp' column
pivot_data4_merged = pivot_data4.merge(merged_data_s_f_filtered, on='timestamp', how='left')

In [40]:
pivot_data4_merged.fillna(0, inplace=True)

In [42]:
pivot_data4_merged_filtered = pivot_data4_merged[pivot_data4_merged['activity_type'] != 0]

In [43]:
pivot_data4_merged_filtered

,timestamp,step_counter,step_detector,accel_x,accel_y,accel_z,gyro_x,gyro_y,gyro_z,linear_accel_x,...,orientation_y,orientation_z,rotation_scalar,rotation_i,rotation_j,rotation_k,rotation_angle,motion_type,motion_encoded,activity_type
0,465,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Stationary,2,Video games
1,465,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Stationary,2,In computer
2,465,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Stationary,2,At home
3,465,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Stationary,2,In computer
4,465,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Stationary,2,Eat
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
129174,466,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Stationary,2,Cooking
129175,466,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Stationary,2,Video games
129176,466,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Stationary,2,In bus
129177,466,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Stationary,2,In vehicle


In [44]:
from sklearn.preprocessing import LabelEncoder

# Create and fit the LabelEncoder
label_encoder_activities= LabelEncoder()
pivot_data4_merged_filtered['activity_encoded'] = label_encoder_activities.fit_transform(pivot_data4_merged_filtered['activity_type'])

/var/folders/4r/5qs7726s3_dc54vtgh1sc2740000gn/T/ipykernel_85916/2909318750.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pivot_data4_merged_filtered['activity_encoded'] = label_encoder_activities.fit_transform(pivot_data4_merged_filtered['activity_type'])


In [45]:
pivot_data4_merged_filtered 

,timestamp,step_counter,step_detector,accel_x,accel_y,accel_z,gyro_x,gyro_y,gyro_z,linear_accel_x,...,orientation_z,rotation_scalar,rotation_i,rotation_j,rotation_k,rotation_angle,motion_type,motion_encoded,activity_type,activity_encoded
0,465,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,Stationary,2,Video games,19
1,465,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,Stationary,2,In computer,5
2,465,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,Stationary,2,At home,0
3,465,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,Stationary,2,In computer,5
4,465,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,Stationary,2,Eat,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
129174,466,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,Stationary,2,Cooking,1
129175,466,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,Stationary,2,Video games,19
129176,466,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,Stationary,2,In bus,4
129177,466,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,Stationary,2,In vehicle,6


In [56]:
pivot_data4_merged_filtered_0 = pivot_data4_merged[pivot_data4_merged['activity_type'] == 0]

In [55]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential, save_model
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import classification_report
from sklearn.utils.class_weight import compute_class_weight
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras import regularizers
import numpy as np

# Split the data into features (X) and target variable (y)
X = pivot_data4_merged_filtered[['accel_x', 'accel_y', 'accel_z', 'gyro_x', 'gyro_y', 'gyro_z', 'orientation_x', 'orientation_y', 'orientation_z', 'motion_encoded', 'timestamp']]
y = pivot_data4_merged_filtered['activity_encoded']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Further split the training data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

# Standardize the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

# Define the model with L1 regularization
model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train_scaled.shape[1],)),
    Dropout(0.5),
    BatchNormalization(),
    Dense(64, activation='relu'),  
    Dropout(0.5),
    BatchNormalization(),
    Dense(len(np.unique(pivot_data4_merged_filtered['activity_encoded'])), activation='softmax')
])

# Compile the model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Encode class labels
le = LabelEncoder()
y_train_encoded = le.fit_transform(y_train)

# Compute class weights using the encoded labels
class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(y_train_encoded), y=y_train_encoded)

# Create a class weight dictionary
class_weight_dict = dict(zip(le.transform(le.classes_), class_weights))

# Train the model with class weights and early stopping
history = model.fit(X_train_scaled, y_train_encoded, epochs=50, validation_data=(X_val_scaled, y_val), class_weight=class_weight_dict)

# Save the model
file_path = "/Users/emilkoch/Desktop/2Tango/messenger/research/Model_Save/activities.keras"
save_model(model, file_path)

# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(X_test_scaled, y_test)
print("Test Accuracy:", test_accuracy)

# Make predictions
y_pred_prob = model.predict(X_test_scaled)
y_pred = np.argmax(y_pred_prob, axis=1)

# Calculate precision, recall, and F1 score
report = classification_report(y_test, y_pred)
print(report)

Epoch 1/50
246/246 [==============================] - 4s 6ms/step - loss: 3.5119 - accuracy: 0.0497 - val_loss: 3.2159 - val_accuracy: 0.0244
Epoch 2/50
246/246 [==============================] - 1s 4ms/step - loss: 3.2503 - accuracy: 0.0413 - val_loss: 3.2697 - val_accuracy: 0.0224
Epoch 3/50
246/246 [==============================] - 1s 5ms/step - loss: 3.1198 - accuracy: 0.0385 - val_loss: 3.2586 - val_accuracy: 0.0183
Epoch 4/50
246/246 [==============================] - 1s 4ms/step - loss: 2.9963 - accuracy: 0.0384 - val_loss: 3.2477 - val_accuracy: 0.0158
Epoch 5/50
246/246 [==============================] - 1s 4ms/step - loss: 3.0380 - accuracy: 0.0388 - val_loss: 3.2512 - val_accuracy: 0.0153
Epoch 6/50
246/246 [==============================] - 1s 4ms/step - loss: 2.8761 - accuracy: 0.0360 - val_loss: 3.2567 - val_accuracy: 0.0127
Epoch 7/50
246/246 [==============================] - 1s 4ms/step - loss: 2.8792 - accuracy: 0.0366 - val_loss: 3.2537 - val_accuracy: 0.0178
Epoch 

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn

In [61]:
from tensorflow.keras.models import load_model, save_model
from sklearn.preprocessing import LabelEncoder

# Load the previously saved model
model = load_model('/Users/emilkoch/Desktop/2Tango/messenger/research/Model_Save/activities.keras')

# Prepare the features from the unlabeled data
X_unlabeled = pivot_data4_merged_filtered_0[['accel_x', 'accel_y', 'accel_z', 'gyro_x', 'gyro_y', 'gyro_z', 'orientation_x', 'orientation_y', 'orientation_z', 'motion_encoded', 'timestamp']]

# Preprocess unlabeled data (assuming you've already standardized the features)
X_unlabeled_scaled = scaler.transform(X_unlabeled)

# Make predictions
y_pred = model.predict(X_unlabeled_scaled)

# Inverse transform the predicted labels
decoded_labels = label_encoder_activities.inverse_transform(np.argmax(y_pred, axis=1))

# Add the decoded labels to the DataFrame
pivot_data4_merged_filtered_0['predicted_activity'] = decoded_labels

5420/5420 [==============================] - 10s 2ms/step


/var/folders/4r/5qs7726s3_dc54vtgh1sc2740000gn/T/ipykernel_85916/666202222.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pivot_data4_merged_filtered_0['predicted_activity'] = decoded_labels


In [62]:
pivot_data4_merged_filtered_0

,timestamp,step_counter,step_detector,accel_x,accel_y,accel_z,gyro_x,gyro_y,gyro_z,linear_accel_x,...,orientation_z,rotation_scalar,rotation_i,rotation_j,rotation_k,rotation_angle,motion_type,motion_encoded,activity_type,predicted_activity
12163,468,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,Stationary,2,0,Sleep
12164,468,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,Stationary,2,0,Sleep
12165,468,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,Stationary,2,0,Sleep
12166,468,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,Stationary,2,0,Sleep
12167,468,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,Stationary,2,0,Sleep
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
185709,1221,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,Stationary,2,0,Took off glasses
185710,1221,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,Stationary,2,0,Took off glasses
185711,1221,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,Stationary,2,0,Took off glasses
185712,1221,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,Stationary,2,0,Took off glasses


We can now predict activities with this time context. With true labels given by users the model can learn iteratively. 

Let's try implement the complementary fusion and activity prediction for the glasses data too. 

In [64]:
glasses = pd.read_csv('/Users/emilkoch/Desktop/2Tango/Data Files/Dataset_2_glasses/glasses.csv')

In [65]:
glasses['accel_x'] = glasses['ACC_X']
glasses['accel_y'] = glasses['ACC_Y']
glasses['accel_z'] = glasses['ACC_Z']
glasses['gyro_x'] = glasses['GYRO_X']
glasses['gyro_y'] = glasses['GYRO_Y']
glasses['gyro_z'] = glasses['GYRO_Z']

In [66]:
glasses = glasses.drop(columns= ['NUM', 'EOG_L', 'EOG_R', 'EOG_H', 'EOG_V', 'ACC_Z', 'ACC_X', 'ACC_Y', 'GYRO_X', 'GYRO_Y', 'GYRO_Z'])

In [67]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

# Exclude the 'timestamp' column from standardization
cols_to_scale = [col for col in glasses.columns if col != 'timestamp']
glasses[cols_to_scale] = scaler.fit_transform(glasses[cols_to_scale])

def complementary_filter(glasses, alpha):
    dt = 0.02  # Assuming a sampling rate of 50 Hz (1/50 = 0.02)
    gyro = glasses[['gyro_x', 'gyro_y', 'gyro_z']].values  # Extract gyro data
    acc = glasses[['accel_x', 'accel_y', 'accel_z']].values  # Extract accelerometer data

    roll_acc = np.arctan2(acc[:, 1], acc[:, 2])  # Calculate roll from accelerometer data
    pitch_acc = np.arctan2(-acc[:, 0], np.sqrt(acc[:, 1] ** 2 + acc[:, 2] ** 2))  # Calculate pitch from accelerometer data

    roll = np.zeros_like(roll_acc)
    pitch = np.zeros_like(pitch_acc)
    yaw = np.zeros_like(gyro[:, 0])

    for i in range(len(acc)):
        roll[i] = alpha * (roll[i] + gyro[i, 0] * dt) + (1 - alpha) * roll_acc[i]
        pitch[i] = alpha * (pitch[i] + gyro[i, 1] * dt) + (1 - alpha) * pitch_acc[i]
        yaw[i] = gyro[i, 2]  # Yaw directly from gyroscope data

    return np.rad2deg(roll), np.rad2deg(pitch), np.rad2deg(yaw)

alpha = 0.98  # Adjust alpha as needed
roll, pitch, yaw = complementary_filter(glasses, alpha)

# Add Euler angles as new columns to the DataFrame
glasses['orientation_x'] = roll
glasses['orientation_y'] = pitch
glasses['orientation_z'] = yaw

In [75]:
import pandas as pd
import numpy as np

def classify_motion_row(row):
    # Extract relevant data from the row
    accel_x = row['accel_x']
    accel_y = row['accel_y']
    accel_z = row['accel_z']
    gyro_x = row['gyro_x']
    gyro_y = row['gyro_y']
    gyro_z = row['gyro_z']
    orientation_x = row['orientation_x']
    orientation_y = row['orientation_y']
    orientation_z = row['orientation_z']
    
    # Classify motion based on accelerometer and gyroscope readings
    if np.any((accel_x != 0) | (accel_y != 0) | (accel_z != 0)):
        # Check if all gyroscope readings are zero
        if not np.any((gyro_x != 0) | (gyro_y != 0) | (gyro_z != 0)):
            # Check orientation for refining moving motion
            if orientation_z > 283.615 or orientation_z < -283.24:
                motion_type = "Vertical Movement"
            elif orientation_x > 12.04 or orientation_x < -11.25:
                motion_type = "Horizontal Movement"
            else:
                motion_type = "Moving"
        else:
            # If gyroscope readings are not all zero, classify as "Turning/Tilting"
            motion_type = "Turning/Tilting"
    elif np.any((gyro_x != 0) | (gyro_y != 0) | (gyro_z != 0)):
        # If accelerometer readings are all zero, check gyroscope readings
        motion_type = "Turning/Tilting"
    else:
        # If both accelerometer and gyroscope readings are zero, classify as "Stationary"
        motion_type = "Stationary"

    return pd.Series([motion_type])

# Apply the function row-wise to the dataframe
glasses['motion_type'] = glasses.apply(classify_motion_row, axis=1)

KeyboardInterrupt: 